In [31]:
import re
import pandas as pd
from datasets import Dataset

# Load Dataset

In [3]:
df = pd.read_csv('../dataset/naruto.csv')
df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [7]:
def remove_paranthesis(text):
    result = re.sub(r"\(.*?\)" , '' , text)
    return result

In [8]:
df["line"] = df['line'].apply(remove_paranthesis)

In [10]:
df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [11]:
df['number_of_words'] = df['line'].str.strip().str.split()
df['number_of_words'] = df['number_of_words'].apply(lambda x : len(x))

In [12]:
df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",25
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [13]:
df['naruto_response_flag'] = 0
df.loc[(df['name'] == 'Naruto') & (df['number_of_words'] > 5) , 'naruto_response_flag'] = 1

In [14]:
df.head()

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",25,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0


In [15]:
df.naruto_response_flag.value_counts()

naruto_response_flag
0    142
1     21
Name: count, dtype: int64

In [22]:
indexes_to_take = list(df[(df['naruto_response_flag'] == 1) & (df.index > 0)].index)
indexes_to_take[:3]

[6, 28, 30]

In [23]:
system_promt = '''Your are Naruto from anime "Naruto". Your responses should reflect his personalist and speech patterns '''

In [26]:
prompts = []

for i in indexes_to_take:
    prompt = system_promt 
    prompt += df.iloc[i - 1]['line']
    prompt += '\n' 
    prompt += df.iloc[i]['line']
    prompts.append(prompt) 
    

In [27]:
prompts[0]

'Your are Naruto from anime "Naruto". Your responses should reflect his personalist and speech patterns   Oh yeah, Naruto!?\n Where’d you come from, Iruka Sensei!? What are you doing here?'

In [30]:
df = pd.DataFrame({'prompt' : prompts})
df.head()

,prompt
0,"Your are Naruto from anime ""Naruto"". Your resp..."
1,"Your are Naruto from anime ""Naruto"". Your resp..."
2,"Your are Naruto from anime ""Naruto"". Your resp..."
3,"Your are Naruto from anime ""Naruto"". Your resp..."
4,"Your are Naruto from anime ""Naruto"". Your resp..."


In [32]:
dataset = Dataset.from_pandas(df)

In [35]:
dataset

Dataset({
    features: ['prompt'],
    num_rows: 20
})